# InsightFace

[论文链接](https://arxiv.org/pdf/1905.00641v2.pdf)

#### 简介
> Nevertheless, in this paper we refer to a broader definition of face localisation which includes face detection, face alignment, pixelwise face parsing and 3D dense correspondence regression.

在这里作者指出他们使用了更广义的人脸定位，包括人脸识别，人脸对齐，像素级人脸捕捉。结合文中图1可以大概得知，InsightFace是一个多任务网络。

#### 损失函数
![](loss_function.png)

在定义loss function的部分，首先是多任务损失，由人脸分类损失、人脸选框回归损失、人脸特征回归损失和密集回归损失（dense regression loss）。这里最后一个损失函数将由之前任务获得的人脸和原人脸进行像素级的比较。作者通过设置$\lambda1, \lambda2, \lambda3$为0.25, 0.1, 0.01来确立更优的人脸选框，人脸特征有比像素级人脸比较有更高的重要性。

#### 执行
C1至C5是预训练的ResNet-152网络的各个阶段，P2至P5是由C1至C5相对应的特征金字塔，P6是C5用3\*3卷积和2步跳跃计算而得的一层。
![](model_overview.png)
<br>

特征金字塔各层尺寸，跳跃和锚点如下：

| Feature Pyramid     | Stride | Anchor              |
|:-------------------:|:------:|:-------------------:|
| P2 (160 × 160 × 256)| 4      | 16, 20.16, 25.40    |
| P3 (80 × 80 × 256)  | 8      | 32, 40.32, 50.80    |
| P4 (40 × 40 × 256)  | 16     | 64, 80.63, 101.59   |
| P5 (20 × 20 × 256)  | 32     | 128, 161.26, 203.19 |
| P6 (10 × 10 × 256)  | 64     | 256, 322.54, 406.37 |

根据我的理解，这里的backbone使用的是预训练的ResNet-152，对应C2至C5的部分。也就是说，大部分的预训练模型都可以被用作backbone。
> For the light-weight model, we can quickly reduce the data size by using a 7 × 7 convolution with stride=4 on the input image, tile dense anchors on P3, P4 and P5 as in *SSH: Single stage headless face detector*, and remove deformable layers. 

作者在推断效率部分也写道，如果使用轻量级模型，如MobileNet-0.25，在7\*7的卷积层和4步跳跃的条件下，数据大小会快速降低，人脸选框也会变得密集。**这里还没有搞懂的点是，如果使用不同于ResNet-152这种有5个阶段的模型，上图中的对应会发生哪些变化？原理又是什么？**但显然，实际操作中InsightFace兼容了许多的的预训练模型，包括我们感兴趣的MobileNet。

# TVM

[论文链接](https://www.usenix.org/system/files/osdi18-chen.pdf)

#### 简介
> TVM is a compiler that exposes graph-level and operator-level optimizations to provide performance portability to deep learning workloads across diverse hardware back-ends.

作者称TVM是一个在多种终端优化深度学习运行性能的集成器。TVM的运行步骤如下图。
![](tvm_overview.png)

#### 特点
* 仅需要几行，用户就可以将已有的模型通过TVM API转变成可部署模块。同时，TVM也支持多种语言，包括C++，Java，和Python。

```python
import tvm as t

graph, params = t.frontend.from_keras(keras_model)
target = t.target.cuda()
graph, lib, params = t.compiler.build(graph, target, params)
```

* TVM使用了operator fusion的方法将多个kernel结合在一起，使得中间的结果不再需要保存。
> Operator fusion combines multiple operators into a single kernel without saving the intermediate results in memory.

* TVM优化了矩阵运算，从而提高运算速度。同时，TVM在后端将嵌套的loop改写为更线性的代码，同样可以提高运算速度。

#### Benchmark
由于论文比较专业，很多术语并不能很好的理解。所以在这里放两张张张作者使用多个主流深度学习库运行多个主流模型的benchmark。运行的平台是ARM Cortex A53 (Quad Core 1.2GHz)，一款常用在移动端的高性能处理器。在实际应用中我们使用的处理器可能比这款处理器要差一些，所以这里更多关注的是使用TVM时处理时间降低的百分比而不是具体的处理时间。

* 从第一张图可以发现，使用TVM优化MobileNet时，相较于TensorFlow Light，**处理时间**由300ms降到了约120ms，下降了60%。
* 从第二张图可以发现，在使用TVM优化ResNet-18时，相较于TensorFlow Light，每个卷积层的**相对处理速度提升**都在2-3倍。
* 作者没有在移动端处理器上比较TVM和MXNet的运行速度，但根据在GPU（NVIDIA Titan X）上的测试结果，MXNet的运行时间为2.3ms而TVM仅需要1ms。
![](benchmarks.png)

#### 结论
TVM是一个非常值得探索的集成器，他的作用类似于TensorFlow，MXNet，可以输入模型，通过优化运算和缓存等方式更快速的输出模型结果。

关于如果使用Python运行TVM还没有仔细研究，这里是一个[教程](https://docs.tvm.ai/tutorials/index.html)的合集，明天继续研究。大家感兴趣的话也可以了解一下。